<a href="https://colab.research.google.com/github/shivakumar-ravichandran/doc_generator/blob/main/doc_generator_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall openai
!pip install openai==0.28

Found existing installation: openai 1.54.4
Uninstalling openai-1.54.4:
  Would remove:
    /usr/local/bin/openai
    /usr/local/lib/python3.10/dist-packages/openai-1.54.4.dist-info/*
    /usr/local/lib/python3.10/dist-packages/openai/*
Proceed (Y/n)? Y
  Successfully uninstalled openai-1.54.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00


In [6]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.0 MB/s eta 0:00:00


# Read the RPG coed file from the local folder or GitHub URL and generate the High level document in docx format.

In [9]:
import os
import requests
import openai
import logging
from datetime import datetime
from docx import Document
from urllib.parse import urlparse


def setup_logging():
    """Sets up logging to a timestamped log file in the logs directory."""
    os.makedirs("logs", exist_ok=True)
    log_file = os.path.join(
        "logs", f"log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    )
    logging.basicConfig(
        filename=log_file,
        level=logging.DEBUG,
        format="%(asctime)s - %(levelname)s - %(message)s",
    )
    return log_file


def read_openai_key(key_file="/content/openai_key.txt"):
    """Reads the OpenAI API key from a file."""
    if not os.path.exists(key_file):
        raise FileNotFoundError(f"The file {key_file} does not exist.")
    with open(key_file, "r") as file:
        key = file.read().strip()
    if not key:
        raise ValueError("The OpenAI API key file is empty.")
    return key

# Function to read and parse the RPG file
def read_rpg_file(file_path):
    """Reads the IBM RPG code file and returns its content line by line."""
    with open(file_path, "r") as file:
        lines = file.readlines()
    return lines


def fetch_rpg_code_from_github(repo_url):
    """Fetches the RPG code from a GitHub repository URL."""
    try:
        logging.info(f"Fetching RPG code from URL: {repo_url}")
        response = requests.get(repo_url)
        response.raise_for_status()
        logging.info("Successfully fetched RPG code.")
        return response.text
    except requests.exceptions.RequestException as e:
        logging.error(f"Failed to fetch RPG code: {e}")
        raise RuntimeError(f"Error fetching RPG code: {e}")


def generate_summary(rpg_code):
    """Generates a detailed summary of the RPG code using OpenAI GPT."""
    try:
        prompt = (
            "You are a technical writer tasked with summarizing legacy IBM RPG code. "
            "Create a high-level summary for the following code, making it understandable "
            "to both technical and non-technical audiences. Ensure the summary includes: "
            "an overview of the code's purpose, key functionalities, and any notable logic.\n\n"
            f"Code:\n{rpg_code}"
        )

        logging.info("Generating summary using OpenAI GPT.")
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct", prompt=prompt, max_tokens=1500, temperature=0.5
        )
        summary = response.choices[0].text.strip()
        logging.info("Summary generation successful.")
        return summary
    except Exception as e:
        logging.error(f"Error generating summary: {e}")
        raise RuntimeError(f"Error generating summary: {e}")


def create_summary_document(summary, output_file="RPG_Summary.docx"):
    """Creates a formatted Word document with the summary."""
    try:
        logging.info(f"Creating Word document: {output_file}")
        doc = Document()
        doc.add_heading("IBM RPG Code Summary", level=1)

        doc.add_paragraph(
            "This document provides a high-level summary of the provided IBM RPG code, "
            "suitable for both technical and non-technical audiences."
        )

        doc.add_heading("Summary", level=2)
        doc.add_paragraph(summary)

        doc.save(output_file)
        logging.info(f"Word document created successfully: {output_file}")
    except Exception as e:
        logging.error(f"Error creating Word document: {e}")
        raise RuntimeError(f"Error creating Word document: {e}")


def main():
    log_file = setup_logging()
    logging.info("Starting RPG code summary generation process.")

    try:
        # Read OpenAI API Key
        openai.api_key = read_openai_key()

        # Fetch RPG code from GitHub
        # github_url = input("Enter the GitHub URL of the RPG file: ").strip()
        # rpg_code = fetch_rpg_code_from_github(github_url)

        # Read RPG file from local folder
        rpg_file_path = "/content/sample_rpg_code.rpg"
        rpg_code = read_rpg_file(rpg_file_path)

        # Generate summary
        summary = generate_summary(rpg_code)

        # Create Word document
        create_summary_document(summary)

        logging.info("Process completed successfully.")
        print(f"Summary document has been created successfully. Check the output file.")
    except Exception as e:
        logging.error(f"Process failed: {e}")
        print(f"An error occurred: {e}")
    finally:
        print(f"Log file saved at: {log_file}")


if __name__ == "__main__":
    main()

Summary document has been created successfully. Check the output file.
Log file saved at: logs/log_20241129_042028.log


# Create a RAG system for the created document.